In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import neurokit2 as nk
import xarray as xr
from bibliotheque import *
from params import *

## PARAMS

In [3]:
srate = down_srate
srate

250

In [4]:
save = True

In [5]:
ecg_interesting_metrics = ['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 'HRV_pNN20', 'HRV_LF', 'HRV_HF', 'HRV_LFHF']
ppg_interesting_metrics = ['mean amplitude', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 'HRV_pNN20', 'HRV_LF', 'HRV_HF', 'HRV_LFHF']
eda_interesting_metrics = ['nb_peaks', 'mean_height', 'EDA_Symp', 'EDA_SympN']

## TOOLS

In [6]:
def eeg_to_metrics(eeg_sig, srate=srate):
    metrics = nk.eeg_power(eeg_sig, sampling_rate=srate)
    return metrics
    
def ecg_to_metrics(ecg_sig, interesting_metrics, srate=srate, get_fci_signal=False):
    peaks, info_ecg = nk.ecg_peaks(ecg_sig, sampling_rate=srate,method='neurokit', correct_artifacts=True)
    R_peaks = info_ecg['ECG_R_Peaks'] # get R time points
    metrics = pd.concat([nk.hrv_time(R_peaks, sampling_rate=srate), nk.hrv_frequency(R_peaks, sampling_rate=srate)], axis = 1)
    
    if get_fci_signal:
        diff_R_peaks = np.diff(R_peaks) 
        x = time_vector(ecg, srate)
        xp = R_peaks[1::]/srate
        fp = diff_R_peaks
        interpolated_hrv = np.interp(x, xp, fp, left=None, right=None, period=None) / srate
        fci = 60 / interpolated_hrv
        
        return metrics[interesting_metrics], fci
    else:
        return metrics[interesting_metrics]
    
def gsr_homemade_metrics(info, dict_symp):   
    nb_peaks = info['SCR_Peaks'].size
    mean_height = np.mean(info['SCR_Height'])
    mean_amp = np.mean(info['SCR_Amplitude'])
    mean_risetime = np.mean(info['SCR_RiseTime'])
    mean_recov_time = np.mean(info['SCR_RecoveryTime'])
    
    data = [nb_peaks, mean_height, mean_amp, mean_risetime, mean_recov_time, dict_symp['EDA_Symp'], dict_symp['EDA_SympN']]
    df = pd.Series(data=data, index = ['nb_peaks', 'mean_height', 'mean_amp', 'mean_risetime', 'mean_recov_time','EDA_Symp','EDA_SympN'])
    return df

def eda_to_metrics(eda_sig, interesting_metrics, srate=srate):
    __ , info = nk.eda_process(eda_sig, sampling_rate=srate, method='neurokit')
    dict_symp = nk.eda_sympathetic(eda_sig, sampling_rate = srate)
    metrics = gsr_homemade_metrics(info, dict_symp)
    return metrics[interesting_metrics]
    
def ppg_amplitude(sig, srate, show=False):
    idx_maxs = signal.find_peaks(sig, distance = srate/2)[0]
    idx_mins = signal.find_peaks(-sig, distance = None)[0]
 
    amplitudes = []
    val_creux = []
    val_sommets = []
    indices_creux = []
    indices_sommets = []
    for idx_sommet in idx_maxs:
        cond = idx_mins < idx_sommet
        if not sum(cond) == 0:
            idx_creux = idx_mins[cond][-1]
            value_creux = sig[idx_creux]
            value_sommet = sig[idx_sommet]
            
            
            val_creux.append(value_creux)
            val_sommets.append(value_sommet)
            indices_sommets.append(idx_sommet)
            indices_creux.append(idx_creux)
        
            amplitude = value_sommet - value_creux
            amplitudes.append(amplitude)
    # print(np.mean(amplitudes))
    

    # df = pd.DataFrame(np.array([np.arange(1,len(val_sommets)+1 , 1), indices_sommets , indices_creux, val_sommets, val_creux, amplitudes]).T, columns = ['cycle','idx_max','idx_min','val_max','val_min','amplitude'])

    if show:
        plt.figure()
        plt.plot(sig)
        plt.plot(indices_sommets, val_sommets, 'x')
        plt.plot(indices_creux, val_creux, 'o')
        plt.show()
        
    return np.mean(amplitudes)

def ppg_to_metrics(ppg_sig, interesting_metrics, srate=srate): 
    peaks = nk.ppg_findpeaks(ppg_sig, sampling_rate=srate, method='elgendi', show=False)
    metrics = pd.concat([nk.hrv_time(peaks, sampling_rate=srate), nk.hrv_frequency(peaks, sampling_rate=srate)], axis = 1)
    amp = ppg_amplitude(ppg_sig, srate)
    metrics.insert(0, 'mean amplitude', amp)
    return metrics[interesting_metrics]

## LOAD DATA

In [7]:
da = xr.load_dataarray('../data_preprocessed/da_cleaned.nc').sel(cleaning = 'clean') # sel already cleaned data

In [8]:
da

<xarray.DataArray (participant: 15, room: 9, dtype: 5, time: 22501)>
array([[[[ 2.84217094e-14,  1.08895065e+01, -3.37755401e+00, ...,
           3.41684774e+01, -8.50098734e+00, -5.32907052e-14],
         [ 0.00000000e+00,  3.05753679e+01,  1.67270879e+01, ...,
           8.46988848e-01, -8.02165826e+01, -7.10542736e-14],
         [-3.99481870e+00, -3.23349538e+00, -2.40230460e+00, ...,
           9.78090929e+00,  5.59578507e+00,  3.78464397e+00],
         [ 1.35440469e+02,  1.35449113e+02,  1.35457694e+02, ...,
           8.01139701e+01,  8.01139069e+01,  8.01138556e+01],
         [ 2.75827751e+01,  2.98107144e+01,  3.18742659e+01, ...,
          -1.12543724e+01, -1.29490419e+01, -1.47835627e+01]],

        [[ 2.70006240e-13,  3.35867958e+01,  1.12906401e+01, ...,
           3.92670513e+01, -5.89494908e+01, -3.48165941e-13],
         [ 2.84217094e-14, -7.16583846e+01,  7.11922899e+01, ...,
          -4.63795439e+02, -4.28517025e+02, -1.27897692e-13],
         [ 9.72328585e-01,  2.62541341e+00,  4.91714400e+00, ...,
           4.97008135e+00,  6.03783369e+00,  7.11076450e+00],
         [ 1.07809219e+02,  1.07891931e+02,  1.07973620e+02, ...,
           1.40340527e+02,  1.40341398e+02,  1.40342099e+02],
         [-2.69893796e-02,  2.01497261e+00,  3.89058492e+00, ...,
...
           2.13447212e+02,  5.13140671e+02,  7.81597009e-14],
         [ 7.10542736e-14, -4.26853966e+01,  3.08607311e+01, ...,
           7.75366759e+01, -1.24481624e+02, -1.42108547e-14],
         [-1.37971192e+00, -1.01666121e+00, -8.17942846e-01, ...,
          -4.68130223e+01, -3.12183358e+01, -1.47638938e+01],
         [ 1.51222263e+02,  1.51230348e+02,  1.51238320e+02, ...,
           1.33332956e+02,  1.33332920e+02,  1.33332893e+02],
         [ 3.91119195e+00,  6.29535336e+00,  8.50622546e+00, ...,
          -1.64113573e+01, -1.79739305e+01, -1.96548977e+01]],

        [[-1.84741111e-13,  2.06771031e+02,  2.59275376e+02, ...,
          -5.47561978e+01, -4.85797263e+01, -9.23705556e-14],
         [ 5.68434189e-14,  1.96192618e+01, -3.26953602e+01, ...,
           5.98148218e+01,  1.32349406e+01,  1.42108547e-14],
         [ 4.02698987e+00,  4.61290739e+00,  5.14984887e+00, ...,
           9.92205147e+00,  1.17204462e+01,  1.35628190e+01],
         [ 1.14702888e+02,  1.14728799e+02,  1.14754523e+02, ...,
           1.19601624e+02,  1.19601607e+02,  1.19601594e+02],
         [-6.26258120e+00, -3.50812536e+00, -9.25309278e-01, ...,
           3.03212115e+00,  1.15212188e+00, -8.81779910e-01]]]])
Coordinates:
  * participant  (participant) object 'P01PPILNI' 'P03PBABCO' ... 'P30BATDI'
  * room         (room) object 'Baseline' 'Take-off' ... 'End of the world'
  * dtype        (dtype) object 'EEGL' 'EEGR' 'ECG' 'EDA' 'PPG'
    cleaning     <U5 'clean'
  * time         (time) float64 0.0 0.004 0.008 0.012 ... 89.99 89.99 90.0 90.0

## DATA TO METRICS

In [97]:
eeg_concat = []
ecg_concat = []
eda_concat = []
ppg_concat = []

for participant in participants:
    print(participant)
    for room in rooms:
        # print(room)
        for dtype in dtypes:
            # print(dtype)
            sig = da.loc[participant, room, dtype, :].values
            
            # plt.figure()
            # plt.plot(sig[0:2500])
            # plt.title(dtype)
            # plt.show()
            
            if not np.mean(sig) == 0: # don't compute metrics if no signal
                if dtype in ['EEGL','EEGR']:
                    metrics = eeg_to_metrics(eeg_sig=sig)
                    metrics.insert(0, 'chan', dtype)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'participant', participant)
                    eeg_concat.append(metrics)
                elif dtype == 'ECG':
                    metrics = ecg_to_metrics(ecg_sig=sig, interesting_metrics = ecg_interesting_metrics)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'participant', participant)
                    ecg_concat.append(metrics)
                elif dtype == 'EDA':
                    metrics = eda_to_metrics(eda_sig=sig, interesting_metrics = eda_interesting_metrics)
                    metrics['room'] = room
                    metrics['participant'] = participant
                    eda_concat.append(metrics)
                elif dtype == 'PPG':
                    metrics = ppg_to_metrics(ppg_sig=sig, interesting_metrics = ppg_interesting_metrics)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'participant', participant)
                    ppg_concat.append(metrics)     
            
eeg_metrics = pd.concat(eeg_concat).drop(columns = ['Channel'])
ecg_metrics = pd.concat(ecg_concat)
eda_metrics = pd.concat(eda_concat, axis = 1).T
ppg_metrics = pd.concat(ppg_concat)

P01PPILNI


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

P03PBABCO


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

P07GHOLE


...

/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

P28JUDGU


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

P30BATDI


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

In [98]:
if save:
    eeg_metrics.to_excel('../metrics/eeg_metrics.xlsx')
    ecg_metrics.to_excel('../metrics/ecg_metrics.xlsx')
    eda_metrics.to_excel('../metrics/eda_metrics.xlsx')
    ppg_metrics.to_excel('../metrics/ppg_metrics.xlsx')

In [9]:
da

<xarray.DataArray (participant: 15, room: 9, dtype: 5, time: 22501)>
array([[[[ 2.84217094e-14,  1.08895065e+01, -3.37755401e+00, ...,
           3.41684774e+01, -8.50098734e+00, -5.32907052e-14],
         [ 0.00000000e+00,  3.05753679e+01,  1.67270879e+01, ...,
           8.46988848e-01, -8.02165826e+01, -7.10542736e-14],
         [-3.99481870e+00, -3.23349538e+00, -2.40230460e+00, ...,
           9.78090929e+00,  5.59578507e+00,  3.78464397e+00],
         [ 1.35440469e+02,  1.35449113e+02,  1.35457694e+02, ...,
           8.01139701e+01,  8.01139069e+01,  8.01138556e+01],
         [ 2.75827751e+01,  2.98107144e+01,  3.18742659e+01, ...,
          -1.12543724e+01, -1.29490419e+01, -1.47835627e+01]],

        [[ 2.70006240e-13,  3.35867958e+01,  1.12906401e+01, ...,
           3.92670513e+01, -5.89494908e+01, -3.48165941e-13],
         [ 2.84217094e-14, -7.16583846e+01,  7.11922899e+01, ...,
          -4.63795439e+02, -4.28517025e+02, -1.27897692e-13],
         [ 9.72328585e-01,  2.62541341e+00,  4.91714400e+00, ...,
           4.97008135e+00,  6.03783369e+00,  7.11076450e+00],
         [ 1.07809219e+02,  1.07891931e+02,  1.07973620e+02, ...,
           1.40340527e+02,  1.40341398e+02,  1.40342099e+02],
         [-2.69893796e-02,  2.01497261e+00,  3.89058492e+00, ...,
...
           2.13447212e+02,  5.13140671e+02,  7.81597009e-14],
         [ 7.10542736e-14, -4.26853966e+01,  3.08607311e+01, ...,
           7.75366759e+01, -1.24481624e+02, -1.42108547e-14],
         [-1.37971192e+00, -1.01666121e+00, -8.17942846e-01, ...,
          -4.68130223e+01, -3.12183358e+01, -1.47638938e+01],
         [ 1.51222263e+02,  1.51230348e+02,  1.51238320e+02, ...,
           1.33332956e+02,  1.33332920e+02,  1.33332893e+02],
         [ 3.91119195e+00,  6.29535336e+00,  8.50622546e+00, ...,
          -1.64113573e+01, -1.79739305e+01, -1.96548977e+01]],

        [[-1.84741111e-13,  2.06771031e+02,  2.59275376e+02, ...,
          -5.47561978e+01, -4.85797263e+01, -9.23705556e-14],
         [ 5.68434189e-14,  1.96192618e+01, -3.26953602e+01, ...,
           5.98148218e+01,  1.32349406e+01,  1.42108547e-14],
         [ 4.02698987e+00,  4.61290739e+00,  5.14984887e+00, ...,
           9.92205147e+00,  1.17204462e+01,  1.35628190e+01],
         [ 1.14702888e+02,  1.14728799e+02,  1.14754523e+02, ...,
           1.19601624e+02,  1.19601607e+02,  1.19601594e+02],
         [-6.26258120e+00, -3.50812536e+00, -9.25309278e-01, ...,
           3.03212115e+00,  1.15212188e+00, -8.81779910e-01]]]])
Coordinates:
  * participant  (participant) object 'P01PPILNI' 'P03PBABCO' ... 'P30BATDI'
  * room         (room) object 'Baseline' 'Take-off' ... 'End of the world'
  * dtype        (dtype) object 'EEGL' 'EEGR' 'ECG' 'EDA' 'PPG'
    cleaning     <U5 'clean'
  * time         (time) float64 0.0 0.004 0.008 0.012 ... 89.99 89.99 90.0 90.0

In [56]:
def ecg_peaks(ecg, srate):
    _,info = nk.ecg_peaks(ecg, sampling_rate=srate)
    return info['ECG_R_Peaks']

def peaks_to_RRI(peaks, srate):
    peaks_time = peaks / srate

    RRIs = []
    for i, time in enumerate(peaks_time):
        if i != 0:
            RRIs.append(time - peaks_time[i-1])
    return np.array(RRIs)*1000

def RRI_to_successive_differences(RRIs):
    successive_differences = []
    for i, RRI in enumerate(RRIs):
        if i != 0:
            successive_differences.append(RRIs[i-1] - RRI))
    return 

def MeanNN(RRIs):
    return np.mean(RRIs)

def SDNN(RRIs):
    return np.std(RRIs)

def RMSSD(RRIs):
    square_of_successive_differences = []
    for i, RRI in enumerate(RRIs):
        if i != 0:
            square_of_successive_differences.append((RRIs[i-1] - RRI)**2)
        
    return np.sqrt(np.mean(square_of_successive_differences))

def pNN50(RRIs):
    return (sum(RRIs) > 50) / RRIs.size


In [11]:
ecg_sig = da.loc['P01PPILNI','End of the world','ECG',:].values

In [45]:
peaks = ecg_peaks(ecg_sig, srate)

In [46]:
RRIs = peaks_to_RRI(peaks, srate)

In [47]:
RRIs

array([980., 688., 700., 680., 704., 696., 696., 680., 712., 708., 688.,
       720., 700., 704., 692., 712., 708., 676., 680., 488., 964., 724.,
       740., 736., 732., 724., 684., 676., 672., 704., 728., 764., 732.,
       708., 692., 672., 676., 660., 692., 856., 956., 960., 988., 956.,
       972., 960., 960., 944., 912., 948., 960., 964., 936., 972., 996.,
       940., 956., 944., 912., 864., 880., 872., 848., 880., 884., 824.,
       872., 864., 812., 824., 804., 788., 768., 788., 756., 740., 760.,
       744., 840., 856., 872., 884., 892., 888., 868., 868., 868., 820.,
       876., 872., 832., 884., 896., 936., 892., 968., 972., 916., 960.,
       936., 884., 924., 912., 864., 920., 932., 880., 924.])

In [42]:
SDNN(RRIs)

109.10220350354018

In [43]:
MeanNN(RRIs)

824.925925925926

In [59]:
nk_metrics = nk.hrv_time(info)
nk_metrics

,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_MCVNN,HRV_IQRNN,HRV_Prc20NN,HRV_Prc80NN,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN
0,824.925926,109.610841,NaN,NaN,NaN,NaN,NaN,NaN,70.785182,71.116347,...,0.168947,209.0,704.0,936.0,15.740741,46.296296,488.0,996.0,13.5,484.375


In [51]:
RMSSD(peaks)

208.0273530803813

In [57]:
pNN50(RRIs)

0.009259259259259259

In [61]:
nk_metrics['HRV_pNN50']

0    15.740741
Name: HRV_pNN50, dtype: float64